In [1]:
%load_ext autoreload
%autoreload 2

import torch as t
import numpy as np

from memory_module import TGNPLMemory
from msg_func import TGNPLMessage
from msg_agg import *

# Test TGNPLMemory

In [2]:
num_nodes = 10
num_prods = 2
raw_msg_dim = 1
state_dim = 10
time_dim = 1
message_module = TGNPLMessage(raw_msg_dim, state_dim+num_prods, time_dim)
aggregator_module = MeanAggregator()

In [3]:
# test initialization
mem = TGNPLMemory(num_nodes,
        num_prods,
        raw_msg_dim,
        state_dim,
        time_dim,
        message_module,
        aggregator_module,
        state_updater_cell="gru",
        use_inventory=True,
        debt_penalty=10,
        consumption_reward=5,
        debug=True)

In [4]:
# before any interactions have been added
np.random.seed(0)
n_id = np.random.choice(num_nodes, size=5, replace=False)
n_id = t.from_numpy(n_id)
print(n_id)
memory, last_update, loss = mem(n_id)

tensor([2, 8, 4, 9, 1])
Total consumed: tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]], grad_fn=<IndexBackward0>)
Total loss: tensor(0., grad_fn=<SumBackward0>)
Total bought: tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]])
msg_s tensor([], size=(0, 38), grad_fn=<CatBackward0>)
msg_d tensor([], size=(0, 38), grad_fn=<CatBackward0>)
msg_p tensor([], size=(0, 38), grad_fn=<CatBackward0>)
aggr tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        

In [6]:
last_update

tensor([0, 0, 0, 0, 0])

In [7]:
# try adding interactions
# expectation: 
# 1) get_updated_memory will print 6 nodes, but memories shouldn't be updated yet
# 2) _update_msg_store should update all three msg stores
src = t.Tensor([0, 0, 1, 2]).long()
dst = t.Tensor([3, 3, 3, 0]).long()
prod = t.Tensor([8, 8, 8, 9]).long()
time = t.Tensor(np.ones(4)).long()
raw_msg = t.Tensor([10, 20, 5, 13]).reshape(-1, 1)
mem.update_state(src, dst, prod, time, raw_msg)

Total consumed: tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]], grad_fn=<IndexBackward0>)
Total loss: tensor(0., grad_fn=<SumBackward0>)
Total bought: tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]])
msg_s tensor([], size=(0, 38), grad_fn=<CatBackward0>)
msg_d tensor([], size=(0, 38), grad_fn=<CatBackward0>)
msg_p tensor([], size=(0, 38), grad_fn=<CatBackward0>)
aggr tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [8]:
mem.msg_s_store

{0: (tensor([0, 0]),
  tensor([3, 3]),
  tensor([8, 8]),
  tensor([1, 1]),
  tensor([[10.],
          [20.]])),
 1: (tensor([1]), tensor([3]), tensor([8]), tensor([1]), tensor([[5.]])),
 2: (tensor([2]), tensor([0]), tensor([9]), tensor([1]), tensor([[13.]])),
 3: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 4: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 5: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 6: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 7: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype

In [9]:
mem.msg_d_store

{0: (tensor([2]), tensor([0]), tensor([9]), tensor([1]), tensor([[13.]])),
 1: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 2: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 3: (tensor([0, 0, 1]),
  tensor([3, 3, 3]),
  tensor([8, 8, 8]),
  tensor([1, 1, 1]),
  tensor([[10.],
          [20.],
          [ 5.]])),
 4: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 5: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 6: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64

In [10]:
mem.msg_p_store

{0: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 1: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 2: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 3: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 4: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 5: (tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], dtype=torch.int64),
  tensor([], size=(0, 1))),
 6: (tensor([], dtype=

In [11]:
# now get memory again - only nodes with interactions should've changed
n_id = t.from_numpy(np.arange(num_nodes))
memory, last_update, loss = mem(n_id)  # test .forward()

Total consumed: tensor([[3.4784, 3.4784],
        [0.5797, 0.5797],
        [1.5073, 1.5073],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000]], grad_fn=<IndexBackward0>)
Total loss: tensor(1057.4208, grad_fn=<SumBackward0>)
Total bought: tensor([[ 0., 13.],
        [ 0.,  0.],
        [ 0.,  0.],
        [35.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]])
msg_s tensor([[-0.2203,  0.0208,  0.1465, -0.1105,  0.1140,  0.0959,  0.1108, -0.0654,
          0.0546,  0.0865,  0.0000,  0.0000, -0.2203,  0.0208,  0.1465, -0.1105,
          0.1140,  0.0959,  0.1108, -0.0654,  0.0546,  0.0865,  0.0000,  0.0000,
         -0.2203,  0.0208,  0.1465, -0.1105,  0.1140,  0.0959,  0.1108, -0.0654,
          0.0546,  0.0865,  0.0000,  0.0000, 10.0000,  0.2286],
        [-0.2203,  0.0208,  0.1465

In [12]:
# 2 and 9, same state, different inventory
# 2 supplied product 9
# product 9 has no inventory
memory[[2,9]]

tensor([[ 0.4034, -0.5517,  0.0473, -0.1073,  0.8412,  0.6808, -0.0140, -0.6415,
          0.0284,  0.9869, -1.5073, -1.5073],
        [ 0.4034, -0.5517,  0.0473, -0.1073,  0.8412,  0.6808, -0.0140, -0.6415,
          0.0284,  0.9869,  0.0000,  0.0000]], grad_fn=<IndexBackward0>)

In [13]:
# 3 and 8, same state, different inventory
# 3 received exactly 35 of product 8
# product 8 has no inventory
memory[[3,8]]

tensor([[ 3.6127e-01, -5.4452e-01,  2.5470e-02, -1.0723e-01,  8.1289e-01,
          6.6089e-01, -3.8717e-02, -6.0877e-01,  1.8932e-02,  9.7933e-01,
          3.5000e+01,  0.0000e+00],
        [ 3.6127e-01, -5.4452e-01,  2.5470e-02, -1.0723e-01,  8.1289e-01,
          6.6089e-01, -3.8717e-02, -6.0877e-01,  1.8932e-02,  9.7933e-01,
          0.0000e+00,  0.0000e+00]], grad_fn=<IndexBackward0>)

In [14]:
# should be unaffected
memory[[4,5,6,7]]

tensor([[-0.2203,  0.0208,  0.1465, -0.1105,  0.1140,  0.0959,  0.1108, -0.0654,
          0.0546,  0.0865,  0.0000,  0.0000],
        [-0.2203,  0.0208,  0.1465, -0.1105,  0.1140,  0.0959,  0.1108, -0.0654,
          0.0546,  0.0865,  0.0000,  0.0000],
        [-0.2203,  0.0208,  0.1465, -0.1105,  0.1140,  0.0959,  0.1108, -0.0654,
          0.0546,  0.0865,  0.0000,  0.0000],
        [-0.2203,  0.0208,  0.1465, -0.1105,  0.1140,  0.0959,  0.1108, -0.0654,
          0.0546,  0.0865,  0.0000,  0.0000]], grad_fn=<IndexBackward0>)

In [15]:
# should only be updated for nodes in transactions
last_update

tensor([1, 1, 1, 1, 0, 0, 0, 0, 1, 1])

## Test attention weight learning

In [16]:
num_nodes = 6
num_prods = 3
raw_msg_dim = 1
state_dim = 2
time_dim = 1
message_module = TGNPLMessage(raw_msg_dim, state_dim+num_prods, time_dim)
aggregator_module = MeanAggregator()

In [17]:
# raw_msg_dim + (3 * memory_dim) + time_dim
message_module.out_channels

17

In [18]:
n_id = t.arange(0, num_nodes).long()
print(n_id)

tensor([0, 1, 2, 3, 4, 5])


In [23]:
mem = TGNPLMemory(num_nodes,
        num_prods,
        raw_msg_dim,
        state_dim,
        time_dim,
        message_module,
        aggregator_module,
        state_updater_cell="gru",
        use_inventory=True,
        debt_penalty=10,
        consumption_reward=5,
        debug=False)
opt = t.optim.Adam(mem.parameters())
for name, param in mem.named_parameters():
    if param.requires_grad:
        print(name, param.data.shape)

time_enc.lin.weight torch.Size([1, 1])
time_enc.lin.bias torch.Size([1])
state_updater.weight_ih torch.Size([6, 17])
state_updater.weight_hh torch.Size([6, 2])
state_updater.bias_ih torch.Size([6])
state_updater.bias_hh torch.Size([6])
output_l.weight torch.Size([2, 2])
output_l.bias torch.Size([2])
input_l.weight torch.Size([2, 2])
input_l.bias torch.Size([2])


In [24]:
t.autograd.set_detect_anomaly(True)
for i in range(1, 31):
#     opt.zero_grad()
    if (i % 2) == 0:
        # 1 sells 5 to 2
        src = t.Tensor([1]).long()
        dst = t.Tensor([2]).long()
        prod = t.Tensor([5]).long()
        time = t.Tensor([i]).long()
        raw_msg = t.Tensor([1]).reshape(-1, 1)
    else:
        # 1 buys 3 and 4 from 0
        src = t.Tensor([0, 0]).long()
        dst = t.Tensor([1, 1]).long()
        prod = t.Tensor([3, 4]).long()
        time = t.Tensor([i, i]).long()
        raw_msg = t.Tensor([2, 4]).reshape(-1, 1)

    print('iter', i)
    mem.update_state(src, dst, prod, time, raw_msg)
    memory, last_update, loss = mem(n_id)
    print('loss', loss)
    prod_emb = mem.memory[mem.num_firms:, :mem.state_dim]
    # prod_emb = t.ones(mem.num_prods, mem.state_dim)
    output_emb = mem.output_l(prod_emb)  # num_products x emb_dim
    input_emb = mem.input_l(prod_emb)  # num_products x emb_dim
    att_weights = output_emb @ input_emb.T  # num_products x num_products
    att_weights = t.nn.ReLU(inplace=False)(att_weights)
    print('att weights', att_weights)
    loss.backward()
    opt.step()

iter 1
loss tensor(125.1899, grad_fn=<SumBackward0>)
att weights tensor([[0.1329, 0.1329, 0.1135],
        [0.1329, 0.1329, 0.1135],
        [0.0869, 0.0869, 0.0669]], grad_fn=<ReluBackward0>)


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [3, 2]], which is output 0 of AsStridedBackward0, is at version 1; expected version 0 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!